In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from glob import glob

import numpy as np
import pandas as pd

In [ ]:
from xray import data, params, trainer, utils

In [ ]:
# source = gcp
filename = "xray_df.csv"

In [ ]:
df = data.get_data_from_gcp(filename)
df.head(3)

In [ ]:
df.drop(
    columns=[
        "Follow-up #",
        "Patient Age",
        "Patient Gender",
        "View Position",
        "OriginalImagePixelSpacing_x",
        "OriginalImagePixelSpacing_y",
        "OriginalImageWidth",
        "OriginalImageHeight",
        "OriginalImage[Width",
        "Height]",
        "OriginalImagePixelSpacing[x",
        "y]",
        "Count_diseases",
    ],
    inplace=True,
    errors="ignore",
)

In [ ]:
df['path'] = df.path.map(lambda x: "/".join(x.split("/")[-3:]))

In [ ]:
df.head(3)

In [ ]:
df_sick = df[df["Enfermo"] != False]
df_sick.shape

In [ ]:
ds_train, ds_val, ds_test = data.split_df(
    dataset=df_sick,
    column_to_filter_by="Patient ID",
    train_val_test=(0.65, 0.15, 0.15),
    total_filter=1,
)
print("train :", ds_train.shape)
print("val :", ds_val.shape)
print("test :", ds_test.shape)

In [ ]:
input_shape = (224, 224)

In [ ]:
filename

In [ ]:
path_to_csv = params.GCP_IMAGE_BUCKET
path_to_csv

In [ ]:
gen_train = data.build_generator(
    img_path=path_to_csv,
    labels_df=ds_train,
    index_col="path",
    labels_col="Fixed_Labels",
    train_set_batch_size=32,
    target_size=(input_shape[0], input_shape[1]),
    binary_class=False,
    test_set=False,
    data_augment=False,
)

gen_val = data.build_generator(
    img_path=path_to_csv,
    labels_df=ds_val,
    index_col="path",
    labels_col="Fixed_Labels",
    train_set_batch_size=32,
    target_size=(input_shape[0], input_shape[1]),
    binary_class=False,
    test_set=False,
    data_augment=False,
)

gen_test = data.build_generator(
    img_path=path_to_csv,
    labels_df=ds_test,
    index_col="path",
    labels_col="Fixed_Labels",
    train_set_batch_size=1,
    target_size=(input_shape[0], input_shape[1]),
    binary_class=False,
    test_set=True,
    data_augment=False,
)
